# Modeling Extremes - Numpyro Pt 2 - MAP

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "" # first gpu
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'FALSE'

import jax
jax.config.update('jax_platform_name', 'cpu')

import numpyro
import multiprocessing

num_devices = multiprocessing.cpu_count()
numpyro.set_platform("cpu")
numpyro.set_host_device_count(num_devices)
jax.config.update("jax_enable_x64", True)

In [2]:
import autoroot
from pathlib import Path
import numpy as np
import xarray as xr
import pandas as pd
import pint_xarray
import arviz as az

from st_evt.viz import plot_histogram, plot_density
from st_evt._src.modules.models.aemet import utils_station
from omegaconf import OmegaConf

import jax
import jax.random as jrandom
import jax.numpy as jnp
import pandas as pd

rng_key = jrandom.PRNGKey(123)

from numpyro.infer import Predictive
import arviz as az

import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS
import xarray as xr
import regionmask

import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FuncFormatter
import seaborn as sns
sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)

%config InlineBackend.figure_format = 'retina'
plt.style.use(
    "https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle"
)

from loguru import logger

# num_devices = 5
# numpyro.set_host_device_count(num_devices)


%matplotlib inline
%load_ext autoreload
%autoreload 2

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


## Paths

In [3]:
results_root_path = "/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/nonstationary_iid_mcmc_redfeten/results"
results_file_name = "nonstationary_iid_mcmc_redfeten.zarr"
results_data_path = Path(results_root_path).joinpath(results_file_name)

figures_path = Path(results_root_path).joinpath("figures/stations")

In [4]:
# results_data_path = "/home/juanjohn/pool_data/dynev4eo/temp/results/scratch_pipelines/results/az_nonstationary_gp_lap_redfeten.zarr"
# figures_path = Path("/home/juanjohn/pool_data/dynev4eo/temp/scratch_pipelines/figures/nonstationary_gp_lap_redfeten")

## Data

In [5]:
DATA_URL = autoroot.root.joinpath("data/ml_ready/aemet/t2max_stations_bm_summer.zarr")
variable = "t2max"
covariate = "gmst"
spatial_dim_name = "station_id"


# LOAD DATA
with xr.open_dataset(DATA_URL, engine="zarr") as f:
    ds_bm = f.load()
    # ds_bm = ds_bm.where(ds_bm.red_feten_mask == 1, drop=True)

### Likelihood Statistics

There are some useful statistics that we can use to evaluate how well our model does.

$$
\begin{aligned}
\text{ELPD WAIC}: && && \\
\text{ELPD WAIC SE}: && && \\
\text{P-Value WAIC}: && && \\
\end{aligned}
$$

In [6]:
variable = "t2max"

## Station Selection

### a - Predetermined Station

In [34]:
from st_evt import AEMET_GOOD_STATIONS, AEMET_BAD_STATIONS

In [35]:
AEMET_BAD_STATIONS["red_feten"]

['3407Y', '9677', 'C018J', 'C426R']

In [36]:
candidate_station = '3407Y' #'3129A' # 

In [37]:
figures_path = figures_path.joinpath(f"{candidate_station}")
figures_path.mkdir(parents=True, exist_ok=True)

### MCMC Results

In [38]:
az_ds = az.from_zarr(store=str(results_data_path))
az_ds_station_pred = az_ds.predictions.sel(station_id = candidate_station)
# num_samples = 5_000
# az_ds_station_pred = az.extract(az_ds, group="predictions", num_samples=num_samples).sel(station_id = candidate_station)
y_data = az_ds.posterior_predictive.sel(station_id = candidate_station)[f"{variable}_true"]
ds_station = ds_bm.sel(station_id = candidate_station)

### EDA Stuff

In [39]:
from st_evt.viz import plot_scatter_ts, plot_histogram, plot_density

In [40]:
utils_station.plot_eda(
    da=ds_station[variable].squeeze(),
    variable_label="2m Max Temperature [°C]",
    # figures_path="./", 
    figures_path=figures_path, 
    figure_dpi=300,
)

2024-12-16 19:47:14.153 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_eda:114 - Plotting BM Data Time Series...
2024-12-16 19:47:14.465 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_eda:130 - Saved Figure:
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/nonstationary_iid_mcmc_redfeten/results/figures/stations/3129A/1391/3407Y/eda/ts_bm_data.pdf
2024-12-16 19:47:14.472 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_eda:132 - Plotting BM Data Histogram...
2024-12-16 19:47:14.710 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_eda:146 - Saved Figure:
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/nonstationary_iid_mcmc_redfeten/results/figures/stations/3129A/1391/3407Y/eda/hist_bm_data.pdf
2024-12-16 19:47:14.712 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_eda:148 - Plotting BM Data Density...
2024-12-16 19:47:15.008 | DEBUG    | st_evt._src.modules.mo

:::{figure}
:label: my-figure
:align: left
:width: 20px

(my-figure-fruit)=
![Here is some fruit 🍏](eda/ts_bm_data.png)

![My vacation pics! 🏝](eda/hist_bm_data.png)

![My vacation pics! 🏝](eda/density_bm_data.png)


Some pictures of fruit and the ocean!
:::

## Posterior Calculations

## Model Inspection

### Trace Plot

In [41]:
variables = [
    "concentration",
    "scale",
    "location_slope",
    "location_intercept",
    ]

utils_station.plot_model_params_critique(
    ds=az_ds_station_pred,
    variables=variables,
    # figures_path="./", 
    figures_path=figures_path, 
    
)

2024-12-16 19:47:26.595 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_model_params_critique:39 - Plotting Parameter Traces...
2024-12-16 19:47:28.187 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_model_params_critique:53 - Saved Figure:
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/nonstationary_iid_mcmc_redfeten/results/figures/stations/3129A/1391/3407Y/params/trace.pdf
2024-12-16 19:47:28.189 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_model_params_critique:55 - Plotting Parameter Jonts...
2024-12-16 19:47:30.764 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_model_params_critique:76 - Saved Figure:
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/nonstationary_iid_mcmc_redfeten/results/figures/stations/3129A/1391/3407Y/params/joint.pdf
2024-12-16 19:47:30.767 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_model_params_critique:77 - Plotting Auto

:::{figure}
:label: my-figure
:align: left
:width: 20px

(my-figure-fruit)=
![Here is some fruit 🍏](params/trace.png)

![My vacation pics! 🏝](params/joint.png)


Some pictures of fruit and the ocean!
:::

## Regression Plot

In [42]:
# calculate model return periods
az_ds_station_pred = utils_station.calculate_ds_return_periods(az_ds_station_pred)

az_ds_quantiles = az_ds_station_pred.quantile(q=[0.025, 0.5, 0.975], dim=["chain", "draw"]).squeeze()
locations = az_ds_station_pred["location"].quantile(q=[0.025, 0.5, 0.975], dim=["chain", "draw"]).squeeze()
scales = az_ds_station_pred["scale"].quantile(q=[0.025, 0.5, 0.975], dim=["chain", "draw"]).squeeze()
return_level_100 = az_ds_station_pred["return_level_100"].quantile(q=[0.025, 0.5, 0.975], dim=["chain", "draw"]).squeeze()
observations = ds_station[variable].squeeze()

In [43]:
utils_station.plot_regression_prediction(
    ds_quantiles=az_ds_quantiles,
    observations=observations,
    # figures_path="./", 
    figures_path=figures_path, 
    covariate=covariate,
    figure_dpi=300,
    y_label="2m Max Temperature, $R_a$ [°C]",
    covariate_label="Global Mean Surface Temperature Anomaly [°C]",
    location_only=True
)

utils_station.plot_regression_prediction(
    ds_quantiles=az_ds_quantiles,
    observations=observations,
    # figures_path="./", 
    figures_path=figures_path, 
    covariate=covariate,
    figure_dpi=300,
    y_label="2m Max Temperature, $R_a$ [°C]",
    covariate_label="Global Mean Surface Temperature Anomaly [°C]",
    location_only=False
)

2024-12-16 19:48:07.583 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_regression_prediction:1021 - Saved Figure:
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/nonstationary_iid_mcmc_redfeten/results/figures/stations/3129A/1391/3407Y/regression/regression_pred_location.png
2024-12-16 19:48:08.131 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_regression_prediction:1021 - Saved Figure:
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/nonstationary_iid_mcmc_redfeten/results/figures/stations/3129A/1391/3407Y/regression/regression_pred.png


## Return Levels

#### Empirical Return Levels

In [44]:
# def calculate_empirical_return_level_gevd_ds(
#     da: xr.DataArray,
#     covariate: str,
#     num_samples: int = 1_000,
#     seed: int = 123,
# ):
#     variable = da.name
#     # # resample array
#     # rng = np.random.RandomState(seed)
#     # y_samples = list(
#     #     rng.choice(
#     #         y_clean.values, size=(y_clean.size), replace=True,
#     #     ) for seed in np.arange(0, num_samples)
#     # )
#     # y_samples = np.stack(y_samples, axis=0)

#     # # expand dims
#     # da = da.expand_dims(dim={"samples": np.arange(0, num_samples)})
#     # # assign coordinates
    
#     # da[variable] = (("samples", covariate), y_samples)
    
#     from st_evt.extremes import calculate_exceedence_probs
#     # add as coordinate
#     da["return_level"] = 1/xr.apply_ufunc(
#         calculate_exceedence_probs,
#         da,
#         input_core_dims=[[covariate]],
#         output_core_dims=[[covariate]],
#         vectorize=True
#     )
#     # swap dimensions
#     # 
#     # da_rl = da_rl.rename("return_level")
#     # print(da_rl)
#     # da_rl = da_rl.rename({covariate: "t2max"})
#     # da_rl[variable] = da[variable]
#     da = da.swap_dims({covariate: variable})
#     return da

In [45]:
# select clean data
y_clean = ds_station[variable].squeeze()

# calculate return period
y_clean = utils_station.calculate_empirical_return_level_gevd_ds(y_clean, covariate=covariate)

# calculate model return periods
az_ds_station_pred = utils_station.calculate_ds_return_periods(az_ds_station_pred)

# Calculate Quantiles
rl_model_quantiles = az_ds_station_pred["return_level"].quantile(q=[0.025, 0.5, 0.975], dim=["chain", "draw"])

2024-12-16 19:48:18.431 | INFO     | st_evt._src.modules.models.aemet.utils_station:calculate_empirical_return_level_gevd_ds:331 - Calculating Return Level...
2024-12-16 19:48:18.436 | INFO     | st_evt._src.modules.models.aemet.utils_station:calculate_empirical_return_level_gevd_ds:339 - Swapping Dims...


### Viz - Return Level 

In [46]:
# fig, ax = model_eval_station.plot_return_periods_ds(
#     rl_model_quantiles=rl_model_quantiles,
#     y=y_clean,
#     covariate=covariate,
#     y_label="2m Max Temperature, $R_a$ [°C]"
# )
# plt.show()

utils_station.plot_return_periods_gmst_ds(
    rl_model_quantiles=rl_model_quantiles,
    y=y_clean,
    covariate=covariate,
    # figures_path="./", 
    figures_path=figures_path, 
    y_label="2m Max Temperature, $R_a$ [°C]"
)

2024-12-16 19:48:22.535 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_gmst_ds:453 - Getting Appropriate Periods...
2024-12-16 19:48:22.537 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_gmst_ds:455 - Intialize Returns...
2024-12-16 19:48:22.537 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_gmst_ds:459 - Creating Data structures...
2024-12-16 19:48:22.539 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_gmst_ds:482 - Plotting...
2024-12-16 19:48:24.158 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_gmst_ds:498 - Saved Figure:
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/nonstationary_iid_mcmc_redfeten/results/figures/stations/3129A/1391/3407Y/returns/returns_gmst_prob_posterior_vs_empirical.png


:::{figure}
:label: my-figure
:align: left
:width: 20px

(my-figure-fruit)=
![Here is some fruit 🍏](returns/returns_gmst_prob_posterior_vs_empirical.png)

Some pictures of fruit and the ocean!
:::

### Viz - 100-Year Return Period

In [47]:
# calculate model return periods
az_ds_station_pred = utils_station.calculate_ds_return_periods(az_ds_station_pred)

# Calculate Quantiles
rl_model_quantiles = az_ds_station_pred["return_level_100"]

In [48]:
# fig, ax = plot_return_periods_100_ds(
#     rl_model_quantiles=rl_model_quantiles,
#     covariate=covariate,
#     x_label="2m Max Temperature, $R_a$ [°C]"
# )
# plt.show()

utils_station.plot_return_periods_100_gmst_ds(
    rl_model=rl_model_quantiles,
    covariate=covariate,
    # figures_path="./", 
    figures_path=figures_path, 
    x_label=r"2m Max Temperature, $R_{100}$ [°C]"
)

2024-12-16 19:48:25.875 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_gmst_ds:609 - Getting Appropriate Periods...
2024-12-16 19:48:25.876 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_gmst_ds:611 - Intialize Returns...
2024-12-16 19:48:25.877 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_gmst_ds:615 - Creating Data structures...
2024-12-16 19:48:25.880 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_gmst_ds:638 - Plotting...
2024-12-16 19:48:26.360 | DEBUG    | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_gmst_ds:654 - Saved Figure:
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/nonstationary_iid_mcmc_redfeten/results/figures/stations/3129A/1391/3407Y/returns/returns_100years_gmst_density.pdf


:::{figure}
:label: my-figure
:align: left
:width: 20px

(my-figure-fruit)=
![Here is some fruit 🍏](returns/returns_100years_gmst_density.png)



Some pictures of fruit and the ocean!
:::

### Viz - 100-Year Return Period Difference

In [49]:
# calculate model return periods
az_ds_station_pred = utils_station.calculate_ds_return_periods(az_ds_station_pred)

# Calculate Quantiles
rl_model = az_ds_station_pred["return_level_100"]
rl_model

<xarray.DataArray 'return_level_100' (chain: 1, gmst: 100, draw: 8000)> Size: 6MB
array([[[20.66562523, 20.60916477, 20.72528081, ..., 20.73608766,
         20.73153436, 20.71214279],
        [20.69892181, 20.64093548, 20.75736623, ..., 20.76507936,
         20.76068437, 20.74193558],
        [20.7322184 , 20.67270428, 20.78945356, ..., 20.79406915,
         20.78983438, 20.77172837],
        ...,
        [23.89544167, 23.69084507, 23.83762746, ..., 23.54822908,
         23.55910814, 23.602043  ],
        [23.92874016, 23.72261387, 23.86971288, ..., 23.57722078,
         23.58826006, 23.63183579],
        [23.96203674, 23.75438458, 23.90180021, ..., 23.60621057,
         23.61741007, 23.66162857]]])
Coordinates:
    alt             float32 4B 766.0
  * chain           (chain) int64 8B 0
  * draw            (draw) int64 64kB 0 1 2 3 4 5 ... 7995 7996 7997 7998 7999
  * gmst            (gmst) float64 800B 0.0 0.02525 0.05051 ... 2.449 2.475 2.5
    lat             float32 4B 40.22
    lon             float32 4B -5.146
    red_feten_mask  uint8 1B 1
    station_id      <U5 20B '3407Y'
    station_name    <U23 92B 'guisando (el risquillo)'

In [ ]:
# fig, ax = plot_return_periods_100_difference_ds(
#     rl_model=rl_model,
#     covariate=covariate,
#     x_label="2m Max Temperature, $R_a$ [°C]",
#     units="[°C]",
#     color="black"
# )
# plt.show()

utils_station.plot_return_periods_100_difference_gmst_ds(
    rl_model=rl_model,
    covariate=covariate,
    # figures_path="./", 
    figures_path=figures_path, 
    x_label="2m Max Temperature, $\Delta R_{100}$ [°C]",
    units="[°C]",
)

2024-12-16 19:48:27.934 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_gmst_ds:784 - Getting Appropriate Periods...
2024-12-16 19:48:27.935 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_gmst_ds:786 - Calculating Difference...
2024-12-16 19:48:27.942 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_gmst_ds:794 - Intialize Returns...
2024-12-16 19:48:27.942 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_gmst_ds:797 - Creating Data structures...
2024-12-16 19:48:27.943 | INFO     | st_evt._src.modules.models.aemet.utils_station:plot_return_periods_100_difference_gmst_ds:817 - Plotting...


:::{figure}
:label: my-figure
:align: left
:width: 20px

(my-figure-fruit)=
![Here is some fruit 🍏](returns/returns_100years_difference_gmst_density.png)

Some pictures of fruit and the ocean!
:::

In [ ]:
# fig, ax = plot_return_periods_100_difference_ds(
#     rl_model=rl_model,
#     covariate=covariate,
#     x_label="2m Max Temperature, $R_a$ [°C]",
#     units="[°C]",
#     color="black"
# )
# plt.show()

utils_station.plot_return_periods_100_difference_prct_gmst_ds(
    rl_model=rl_model,
    covariate=covariate,
    # figures_path="./", 
    figures_path=figures_path, 
    x_label="2m Max Temperature, $\Delta R_{100}$ [%]",
)

:::{figure}
:label: my-figure
:align: left
:width: 20px

(my-figure-fruit)=
![Here is some fruit 🍏](returns/returns_100years_difference_prcnt_gmst_density.png)


Some pictures of fruit and the ocean!
:::